In [1]:
import torch
import torch.nn as nn
from torchvision import models
from transformers import DistilBertModel
from test_harness import test_harness

class PhishingClassifier(nn.Module):
    def __init__(self):
        super(PhishingClassifier, self).__init__()
        self.cnn = models.resnet18(pretrained=True)
        self.cnn_fc_features = self.cnn.fc.in_features
        self.cnn.fc = nn.Identity() 
        
        # Pre-trained DistilBERT for URL feature extraction
        self.bert = DistilBertModel.from_pretrained('distilbert-base-uncased')
        
        # Classifier combining both CNN and BERT features
        self.classifier = nn.Sequential(
            nn.Linear(self.cnn_fc_features + self.bert.config.hidden_size, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 2)
        )

    def forward(self, url_input_ids, url_attention_mask, image):
        image_features = self.cnn(image)
        
        bert_outputs = self.bert(input_ids=url_input_ids, attention_mask=url_attention_mask)
        url_features = bert_outputs.last_hidden_state[:, 0, :]
        
        combined_features = torch.cat((image_features, url_features), dim=1)
        
        logits = self.classifier(combined_features)
        return logits
    
    def test_name(self):
        return 'cnn_with_url'

device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
model = PhishingClassifier().to(device)
test_harness(model)
torch.save(model.state_dict(), f"{model.test_name()}_phishing_classifier.pt")



/opt/homebrew/Caskroom/miniforge/base/envs/phishing-edge/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/phishing-edge/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/opt/homebrew/Caskroom/miniforge/base/envs/phishing-edge/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, 

Epoch 1/10:   0%|          | 0/7080 [00:00<?, ?it/s]

RuntimeError: Boolean value of Tensor with more than one value is ambiguous